In [2]:
import requests
from bs4 import BeautifulSoup
import time, random
import re

from selenium.webdriver.support.expected_conditions import none_of

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_0) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
}

year=2026
years_list=[]
list_href_mvp=[]
list_href_player=[]
base_url = "https://www.basketball-reference.com"
mvp_url="/awards/mvp.html"
full_url=base_url+mvp_url

response = requests.get(full_url, headers=headers)
response.encoding = "utf-8"
print(response.status_code)
soup = BeautifulSoup(response.text, "html.parser")

rows = soup.select("tbody tr")
count=0

for row in rows:
    if count== 7:
        break
    cells = row.select("td")
    third_td = cells[2]
    link_tag = third_td.select_one("a")
    list_href_mvp.append(link_tag["href"])
    # print(link_tag["href"])



    count += 1
for best_mvp_each_year_url in list_href_mvp:
    full_url_mvp_each_year = base_url + best_mvp_each_year_url

    response = requests.get(full_url_mvp_each_year, headers=headers)
    response.encoding = "utf-8"
    print(response.status_code)
    time.sleep(random.uniform(3,5))
    soup_mvp_person = BeautifulSoup(response.text, "html.parser")

    players = soup_mvp_person.select("#div_mvp tbody tr")
    year=year-1
    years_list.append(year)
    for player in players:
        player_cell = player.select("td")
        link_player = player_cell[0].select_one("a")

        age=player_cell[1].text
        age_int=int(age)

        rank = player.select("th")
        player_rank = rank[0].text
        numbers = "".join(re.findall(r"\d+", player_rank))
        number_int = int(numbers)
        list_href_player.append({"year": year, "url_player": link_player["href"], "age": age_int, "player_rank": number_int})






200
200
200
200
200
200
200
200


In [14]:
import pandas as pd
positions = []
records=[]
height_cm=0
weight_kg=0
experience_years=0
cleaned=None
player_name=None
team_name=None

for player in list_href_player:
    response = requests.get(base_url+player["url_player"],headers=headers)
    response.encoding = "utf-8"
    info_player_soup = BeautifulSoup(response.text, "html.parser")


    url = player["url_player"]  # just a string
    player_id = re.search(r'([^/]+)\.html$', url).group(1)


    player_name_elem = info_player_soup.select_one("#meta h1")
    player_name = player_name_elem.get_text(strip=True) if player_name_elem else None

    final_clean_positions = []
    position_elem = info_player_soup.select_one("#meta p:has(strong:-soup-contains('Position'))")

    text = position_elem.get_text(" ", strip=True)
    position = text.split("Shoots:")[0].replace("Position:", "").strip()
    clean_position = position.replace(", and", ",")
    clean2_position = clean_position.replace(" and ", ",")
    positions = [p.strip() for p in clean2_position.split(",") if p.strip()]
    for p in positions:
        cleaned = p.strip()
        cleaned = cleaned.replace("▪", "")
        cleaned = cleaned.strip()
        final_clean_positions.append(cleaned)

    team_strong = info_player_soup.select_one("#meta strong:-soup-contains('Team')")
    if team_strong:
        team_link_tag = team_strong.find_next("a")
        if team_link_tag:
            team_name = team_link_tag.get_text(strip=True)
    else:
        team_name = None



    exp_strong = info_player_soup.select_one("#meta strong:-soup-contains('Experience')")
    if exp_strong:
        experience_text = exp_strong.next_sibling.strip()
        match = re.search(r"\d+", experience_text)
        experience_years = int(match.group(0))

    else:
        exp_strong = info_player_soup.select_one("#meta strong:-soup-contains('Career Length')")
        experience_text = exp_strong.next_sibling.strip()
        match = re.search(r"\d+", experience_text)
        experience_years = int(match.group(0))




    height_weight_elems = info_player_soup.select_one("#meta p:has(span:-soup-contains('lb'))")
    text = height_weight_elems.get_text(" ", strip=True)
    match = re.search(r"\((\d+)cm,\s*(\d+)kg\)", text)
    height_cm = int(match.group(1))
    weight_kg = int(match.group(2))

    records.append({
        "player_id": player_id,
        "player_name": player_name,
        "age": player["age"],
        "rank": player["player_rank"],
        "season": player["year"],
        "height_cm": height_cm,
        "weight_kg": weight_kg,
        "experience_years": experience_years,
        "positions": final_clean_positions,
        "team": team_name,
        "total_point": None,
        "groupName":"MJT"
    })

    time.sleep(random.uniform(3,5))
df = pd.DataFrame(records)
print(df.head())


gilgesh01
jokicni01
antetgi01
tatumja01
mitchdo01
jamesle01
cunnica01
edwaran01
curryst01
brunsja01
hardeja01
mobleev01
jokicni01
gilgesh01
doncilu01
antetgi01
brunsja01
tatumja01
edwaran01
sabondo01
duranke01
embiijo01
jokicni01
antetgi01
tatumja01
gilgesh01
mitchdo01
sabondo01
doncilu01
curryst01
butleji01
foxde01
brunsja01
moranja01
jokicni01
embiijo01
antetgi01
bookede01
doncilu01
tatumja01
moranja01
curryst01
paulch01
derozde01
duranke01
jamesle01
jokicni01
embiijo01
curryst01
antetgi01
paulch01
doncilu01
lillada01
randlju01
rosede01
goberru01
westbru01
simmobe01
hardeja01
jamesle01
leonaka01
antetgi01
jamesle01
hardeja01
doncilu01
leonaka01
davisan02
paulch01
lillada01
jokicni01
siakapa01
butleji01
tatumja01
antetgi01
hardeja01
georgpa01
jokicni01
curryst01
lillada01
embiijo01
duranke01
leonaka01
westbru01
goberru01
jamesle01
   player_id              player_name  age  rank  season  height_cm  \
0  gilgesh01  Shai Gilgeous-Alexander   26     1    2025        198   
1  jokicni01  

In [15]:

df.to_csv("mvp.csv", index=False)